<a href="https://colab.research.google.com/github/FerminMendez/ModuleAI/blob/main/Statistic/MultipleRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 Importando los datos


Primeramente importamos algunas de las librerias que vamos a usar. Principalmente enfocadas ala manejo de datos y gráficas.

In [2]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


Accedemos al path donde está el Data Set. En este caso tenemos información financiera de las empresas de EU a partir del año 2000;

In [ ]:
drive.mount("/content/gdrive")
!pwd
%cd "/content/gdrive/MyDrive/DataSetIA/US_dataset"
!ls

In [ ]:
data = pd.read_csv('dataus2023.csv')
data.head()

In [ ]:
firms = pd.read_csv('firmsus2023.csv')
firms.head()

In [ ]:
firms_selected=firms[["empresa","Nombre","status","naics1"]]
firms_selected.columns=['firm',"Empresa","status","industria"]
data =firms_selected.merge(data, on='firm', how='left')
data['qy'] = data['q'].str[:4]
data['qn'] = data['q'].str[5]
data.head()

Como se puede ver tenemos dos DataSet que se relacionan. Con el fin de tener toda la información relevante en un dataset vamos a mezclar los dataframe de data y firms.

## 0.1 Tranformando los tipos de datos

In [ ]:
data['qdate']=pd.PeriodIndex(data.q, freq='Q')

## 0.2 Más informacion relevante


In [ ]:
data['ry']= np.log(data['adjprice']) - np.log(data.groupby(['firm'])['adjprice'].shift(4))
data['rq']= np.log(data['adjprice']) - np.log(data.groupby(['firm'])['adjprice'].shift(1))
# ebit = revenue - cogs - sgae
data['ebit']=data['revenue'] - data['cogs'] - data['sgae']

#opm = ebit / revenue
# data['opm']=data['ebit']/ data['revenue'] -> Hay que hacer las validaciones de que el divisor sea distinton de 0
data['opm']= np.where(data['revenue']==0,np.NaN, data['ebit']/ data['revenue'])
# Calculate net income as: netincome = ebit + otherincome + extraordinaryitems - financial expenses - incometax
data['netincome'] = data['ebit']+ data['otherincome'] +data['extraordinaryitems'] - data['finexp'] -data['incometax']

#Calculate profit margin (ratio) as: pm = ni / revenue
data['pm'] = np.where(data['revenue']==0,np.NaN, data['netincome']/ data['revenue'])

#Calculate asset turn over ratio: ato = revenue / totalassets
data['ato'] = np.where(data['totalassets']==0,np.NaN, data['revenue']/ data['totalassets'])

#Calculate acid ratio: acidratio = currentassets / currentliabilities
data['acidratio']= np.where(data['currentliabilities']==0,np.NaN, data['currentassets']/ data['currentliabilities'])

#Calculate financial leverage ratio as: finlev=longdebt / totalassets
data['finlev'] = np.where(data['totalassets']==0,np.NaN, data['longdebt']/ data['totalassets'])

#Calculate market value as: mvalue = originalprice * sharesoutstanding
data['mvalue']=data['originalprice']*data['sharesoutstanding']

#Calculate book value as: bookvalue = totalassets - totalliabilities
data['bookvalue'] = data['totalassets'] - data['totalliabilities']

#Retornos futuros a 1 y 4 trimestres
data['f1rq']= data.groupby(['firm'])['rq'].shift(-1)
data['f4rq']= data.groupby(['firm'])['rq'].shift(-4)

#Retornos futuros a 1 y 4 años
data['f1ry']= data.groupby(['firm'])['ry'].shift(-1)
data['f4ry']= data.groupby(['firm'])['ry'].shift(-4)

#Separción del número de trimestre y año
data['qy'] = data['q'].str[:4]
data['qn'] = data['q'].str[5]

In [ ]:
#data

In [ ]:
data.columns

Copia de los datasets y subdatasets que usaremos


In [ ]:
#Dataframe general
#data: Este dataset tiene TODA la información original de los datasets originales, además los cálculos financieros.
data=data.copy()

#Dataframe manofactura
#df_m:  Filtro por indudria. Año cierre del año fiscal 2022
df_m=data.copy()
df_m=df_m[(df_m['fiscalmonth']==12) & (df_m['industria']=="Industrias manufactureras") ]


#Dataframe manofactura año 2022
df=data.copy()
df=df[(df['fiscalmonth']==12) & (df['industria']=="Industrias manufactureras") & df['qy']=='2022']
df_m_2022=df;


#1 Estadistica avanzada


## 1.1 Analitica descriptiva


### Análisis de los ratios: Comparando media ,media y mediana.





In [ ]:
#df_m.describe()

Preparamos el dataset para trabajar con los ratios

In [ ]:
#Trabajamos con el dataset de manofactura
df=df_m.copy()
#Selección de las columnas que corresponden a un ratio, sus denominadores y numeradores
ratios_columns=     ['opm'    ,'pm'        ,'ato'         ,'acidratio'         ,'finlev' ]
numerators_columns= ['ebit'   ,'netincome' ,'revenue'     ,'currentassets'     ,'longdebt']
denomintaor_columns=['revenue','revenue'   ,'totalassets' ,'currentliabilities','totalassets']

Con la ayuda de .decribe() vamos a encontrar la media y mediana de los ratios. Los pesos ponderados los calcularemos de manera manual y desplegaremos toda la información de los ratios en un mismo dataframe.

In [ ]:
df_stats=df[ratios_columns].describe().T[['mean','50%','std']].T
weight_mean=np.zeros(len(ratios_columns), dtype = float)
for i in range(len(ratios_columns)):
  name=ratios_columns[i]
  weight_mean[i]=df[numerators_columns[i]].sum()/df[denomintaor_columns[i]].sum()

df_stats=df_stats.T
df_stats['weight mean']=weight_mean
print(df_stats)

Hasta ahora hemos calculado la media, mediana y media ponderada de la industria manofacturera. ¿Qué medida es la más representativa?¿Qué podemos decir del profit margin y asset turn over de la industria manofaturera?

La media es el mejor punto de referencia cuando tenemos una distribución de datos normales. En estos casos la media y mediana suelen ser muy parecidas. En este caso tenemos todo lo contrario. Una dispersión de datos muy grande, la deviación estandar se aleja mucho de lo limites y no podemos decir que alguna de estas medidas represente el comportamiento de la industria manofacturera en los últimos años.

Cuando utilizamos **la media ponderada** estamos tratando a la industria manofacturera como una sola entidad, no busca describir el comportamiento de una empresa promedio o típica, describe el comportamiento del sector, **es la mejor medida de tendencia cuando tenemos empresas tan pequeñas o tan grandes**, de esta manera el peso de cada empresa corresponde al poder explicativo que tiene sobre el comportamiento del sector.

**Profit Margin de la industria manofacturera**
*   pm = 0.074770

*Tenemos que de todos los ingresos de la industria manofacturera en los últimos 20 años se capitaliza el 7.4% de los ingresos como ganancias.*

**Asset turn over de la industria manofacturera**

*   ato = 0.743507

*Tenemos que el ato es la capacidad de una empresa para convertir sus activos en ganancias. De acuerdo a esta información la industria manofacturera en un periodo de un año ha logrado transformar el 74% del valor de sus activos en ganancias en un año durante los útltimos 20 años.*

**Con esto podemos decir que la industria manofacturera desde el año 2000 ha mostrado una buena salud financiera**


In [ ]:
#Replicamos el proceso para el año 2022
df=df_m_2022.copy()
ratios_columns=     ['opm'    ,'pm'        ,'ato'         ,'acidratio'         ,'finlev' ]
numerators_columns= ['ebit'   ,'netincome' ,'revenue'     ,'currentassets'     ,'longdebt']
denomintaor_columns=['revenue','revenue'   ,'totalassets' ,'currentliabilities','totalassets']
df_stats=df[ratios_columns].describe().T[['mean','50%','std']].T
weight_mean=np.zeros(len(ratios_columns), dtype = float)
for i in range(len(ratios_columns)):
  name=ratios_columns[i]
  weight_mean[i]=df[numerators_columns[i]].sum()/df[denomintaor_columns[i]].sum()
df_stats=df_stats.T
df_stats['weight mean']=weight_mean
print(df_stats)

2022:


*   pm = 0.104924
*   ato = 0.738371



Haciendo el mismo análisis con solo el año 2022 Podemos ver que la capacidad de hacer activos gananacias es casi la misma. Pero a comparación del registro históricon en el año 2022 el margen de ganancias está 3% por encima. **Es decir el año 2022 ha sido un buen años financieramente hablando para la industria manofacturera.**

##1.2 Regresión lineal multiple

### 1.2.1 Calculo de variables para la regresión



#### Clasificación por tamaño
Agregaremos etiquetas de tamaño a cada empresa. Las clasificaremos en pequeña mediana y grande de acuerdo al percentil en el que se encuentren. Vamos a hacer una partición en 3 partes iguales con los cuartiles 33% y 66%.

In [ ]:
df=df_m.copy();
# Calculate percentiles
limits=[0.3333,0.6666]
percentiles = df['mvalue'].quantile(limits)

# Define function to categorize values
def categorize_size(value):
    if value <= percentiles[limits[0]]:
        return 'small'
    elif value <= percentiles[limits[1]]:
        return 'medium'
    else:
        return 'big'

# Apply the categorize_size function to create the "size" column
df['size'] = df['mvalue'].apply(categorize_size)


In [ ]:
# Create a histogram with colors based on 'size'
sns.histplot(data=df, x='mvalue', bins=80, log_scale=(False, True), hue='size', multiple='stack')

plt.xlabel('Market Value')
plt.ylabel('Frequency (log scale)')
plt.title('Logarithmic Histogram of Market Value by Size')

plt.tight_layout()
plt.show()


Comprobamos que tengan tamaños similares después de hacer el One-hot-encoding con size

#### Dummie encoding por tamaño de la empresa

Vamos a obtener columnas adicionales donde nos indica si es de tamaño chica, mediana o grande.

IMPORTANTE: Por fines prácticos calculamos los 3 dummies, uno por cada tamaño. Pero en los siguientes pasos solo usaremos lo dummies de big y small ya que vamos a hacer el análisis de efecto indirecto tomando como grupo base aquellas empresas que son de tamaño mediano.

In [ ]:
#Dummie encoding
df = pd.get_dummies(df, columns=['size'], prefix='size')
df[['size_big','size_medium','size_small']].describe()

#### Calculando otras variables

*   Calculate operating earnings per share deflated by stock price: oepsp = (ebit / sharesoutstanding) / originalprice.
*   Calculate earnings per share deflated by stock price: epsp = (netincome / sharesoutstanding) / originalprice
*   Calculate book-to-market ratio: bmr = bookvalue / marketvalue

**Tenemos que oepsp=  (ebit / sharesoutstanding) / originalprice = (ebit / (originalprice * sharesoutstanding)) = ebit / mvalue**

**De igual manera epsp =  (netincome / sharesoutstanding) / originalprice = netincome / mvalue**

In [ ]:
df['oepsp']=np.where(df['mvalue']==0,np.NaN, df['ebit']/ df['mvalue'])
df['epsp']=np.where(df['mvalue']==0,np.NaN, df['netincome']/ df['mvalue'])
df['bmr']=np.where(df['mvalue']==0,np.NaN, df['bookvalue']/ df['mvalue'])


Este dataset en un principio contenia la información de la industria manofacturera en todos los trimestres. Ahora hemos agregado los dummies por tamaño y otras variables extra. Vamos a guardar este dataset como df_regression.

In [ ]:
df_regression=df
df.describe()

### Modelo de regresión para predecir el retorno anual
**Variable dependiente:**

*   Retorno anual ('f1ry')

**Variables independientes**

*   Size dummies ('size_big','size_small')
*   Operating earnings per share deflated by stock price ('oepsp')
*   Earnings per share deflated by stock price ('epsp')
*   BookValue-MarketValue Ration ('bmr)


#### Filtro de solo las variables que vamos a utilizar para esta regresión.

In [ ]:
df=df_regression.copy()
analized=['size_big','size_small','pm','ato','oepsp','epsp','bmr','f1ry']
df[analized]
df.describe()


#### Versión 1 : Regresión (EXTRA)
Antes de tratar las variables vamos a correr una primera versión de la regresión para ver cómo va evolucionando

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df[['size_big','size_small','pm','ato','oepsp','epsp','bmr']]
# Add a constant term (intercept) to the features
X = sm.add_constant(X)
X.fillna(X.mean(), inplace=True)
# Define your target variable (dependent variable)
y = df["f1ry"]
# Create and fit the linear regression model
model = sm.OLS(y, X, missing="drop").fit()

In [ ]:
# Calculate the VIF for each independent variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# Print the VIF values
print(vif_data)

In [ ]:
# Get the model summary
model_summary = model.summary()
# Print the model summary
print(model_summary)

RESUMEN:
*   Una R^2 de 0.15. Es un modelo malo.
*   Detectamos alta colinealidad en algunas variables.
*    Tenemos T values significativos en todas las variables a excepción de pm y ato.

Ideas:
*   Que sea un empresa grande es significativo para el aumento de la variable dependiente. Las empresas grandes tienen mayores retornos trimestrales proyectados a un año.
*   Analogo, que sea una empresa pequeña es significativo para tener menores retornos.




### 1.2.2 Winsorización


#### Obtenemos el dataset base para la regresión y filtramos las variables que vamos a utilizar.

Estadistica descriptiva

In [ ]:
df=df_regression.copy()
analized=['size_big','size_small','pm','ato','oepsp','epsp','bmr','f1ry']
df=df[analized]
df.describe()

In [ ]:
# Create histograms
var2plot=['pm','ato','oepsp','epsp','bmr']
plt.figure(figsize=(15, 10))
for i, column in enumerate(var2plot):
    plt.subplot(2, 3, i + 1)
    plt.hist(df[column], bins=20, color='skyblue', alpha=0.75)
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')

# Adjust spacing between subplots
plt.tight_layout()
plt.show()

#### Winsorización manual
Vamos a aplicar una winsorización manual. Es decir vamos a personalizar el porcentaje que se va a extraer de cada variable. Con la estádistica descriptiva anterior vamos a buscar valores que mejoren el error estándar. Notese que tenemos medias no mayores a 100 en todos los casos y desciacipnes estándares que llegan hasta los 3000. veamos cómo podemos mejorar estas desviaciones con winsorización.

**Guardamos el dataset winsorizado como df_winsorized**

In [ ]:
df=df_regression.copy()
analized=['size_big','size_small','pm','ato','oepsp','epsp','bmr','f1ry']
df=df[analized]
columns_winzorization=['pm','ato','oepsp','epsp','bmr']
low_quartile=         [0.02, 0.01,  0.01,  0.01,  0.01]
high_quartile=        [0.99, 0.99,  0.99,  0.99,  0.99]
for i in range(len(columns_winzorization)):
  percentile_1 = df[columns_winzorization[i]].quantile(low_quartile[i])
  percentile_99 = df[columns_winzorization[i]].quantile(high_quartile[i])
  df[columns_winzorization[i]]=df[columns_winzorization[i]].clip(lower=percentile_1, upper=percentile_99)

df_winsorized=df
df.describe()

En este caso vemos que con la winsorización de 1%-99% funciona bastante bien. Hacer 2% en PM mejora std de 25 a 12.

In [ ]:
# Create histograms
var2plot=['pm','ato','oepsp','epsp','bmr']
plt.figure(figsize=(15, 10))
for i, column in enumerate(var2plot):
    plt.subplot(2, 3, i + 1)
    plt.hist(df[column], bins=20, color='skyblue', alpha=0.75)
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')

# Adjust spacing between subplots
plt.tight_layout()
plt.show()

Si bien winsorizar 1%-99% No resolvió todos los problemas con datos extremos mejoró mucho la escala. A diferencia los histogramas sin winsorizar esta escala redujo significativamente el número de cifras de los valores extremos. Por ejemplo de esps de -3,000,000 a -2.5 o de -25,000 a -200.


### 1.2.3 Multicolinealidad


#### Matriz de correlación y VIF

In [ ]:
correlation_matrix = df.corr()
# Set up the heatmap figure
plt.figure(figsize = (20, 10))
# Create heatmap using seaborn
sns.heatmap(correlation_matrix, annot = True, cmap = "coolwarm", center = 0)
# Show the plot
plt.title("Correlación de las empresas en la industria manofacturera")
plt.show()

Vemos que esps y oesps tienen un alto índice de correlación. Básicamente explican lo mismo en el modelo de regresión.

In [ ]:
X = df[['size_big','size_small','pm','ato','oepsp','epsp','bmr']]
# Add a constant term (intercept) to the features
X = sm.add_constant(X)
X.fillna(X.mean(), inplace=True)
# Calculate the VIF for each independent variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Print the VIF values
print(vif_data)

Cómo era de esperarse tenemos que oespsp y espsp tienen alta relación. Para mejorar el modelo vamos a eliminar una variable en este caso oespsp. Para eliminarla.

In [ ]:
df_winsorized_multicol = df.drop(["oepsp"], axis=1)

###1.2.4 Multiple regression model

#### Versión 2: Regresión después de winsoerizar y tratar multicolinealidad.



In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
df=df_winsorized_multicol.copy()
#Solo las variables que nos importan
selected=['size_big','size_small','pm','ato','epsp','bmr']
X = df[selected]
# Add a constant term (intercept) to the features
X = sm.add_constant(X)
X.fillna(X.mean(), inplace=True)
# Define your target variable (dependent variable)
y = df["f1ry"]
# Create and fit the linear regression model
model = sm.OLS(y, X, missing="drop").fit()
# Calculate the VIF for each independent variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# Print the VIF values
print(vif_data)

In [ ]:
# Get the model summary
model_summary = model.summary()
# Print the model summary
print(model_summary)

RESUMEN:
*   Una R^2 de 0.25 mejor que el 0.15 que obtuvimos antes de winsorizar.
*   Ya no ha alta colinealidad entre nuestrasa variables.
*   En este caso TODAS nuestras variables tienen |t values| mayores que 2. TODAS NUESTRAS VARIABLES SON SIGNIFICATIVAS para explicar el modelo.

Ideas:
*   A expeción de pm, ningún intervalo del 95% de confianza contiene el 0. Son mayores o menores que 1. Por lo que podemos decir que existen ciertos factores que afectan para aumentar los retornos trimestrales el próximo año.

Una empresa aumenta los retornos con evidencia estadistica si:
1.   Es una empresa grande en lugar de una empresa mediana.
2.   Aumenta su asset turn over ratio.
3.   Aumenta su earnings per share deflated by stock price.

Una empresa decrese los retornos con evidencia estadistica si:
1.   Es una empresa pequeña en lugar de una empresa mediana.
2.   Aumenta el ratio de valor de libro sobre valor de mercado.



#### Versión 3: Regresión con winsorización, colinealidad tratada y variables de interacción



Añadimos terminos de interacción
Para este modelo agregamos la interacción de earnings per share deflated by stock price con el tamaño de la empresa. Queremos comprobar que la etiqueta por tamaño de la empresa y esta variable explican mejor el modelo.

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
df=df_winsorized_multicol.copy()
#Añadimos variables de interacción
df['epsp*big']=df['epsp']*df['size_big']
df['epsp*small']=df['epsp']*df['size_small']
#Seleccionamos las variables para el modelo
selected=['size_big','size_small','pm','ato','epsp','epsp*big','epsp*small','bmr']
X = df[selected]
# Add a constant term (intercept) to the features
X = sm.add_constant(X)
X.fillna(X.mean(), inplace=True)
# Define your target variable (dependent variable)
y = df["f1ry"]
# Create and fit the linear regression model
model = sm.OLS(y, X, missing="drop").fit()
# Calculate the VIF for each independent variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# Print the VIF values
print(vif_data)

In [ ]:
# Get the model summary
model_summary = model.summary()
# Print the model summary
print(model_summary)

**INTERPRETEMOS ESTE MODELO CON VARIABLESD DE INTERACCION**
RESUMEN:
*   Una R^2 de 0.253 casi igual que 0.252 que obtuvimos antes, no presenta mejora significativa al tratar de predecir el modelo. Solo explica el 25% de la variabilidad del modelo.
*   Hay alta colinealidad entre la variable de interacción de empresa pequeña con su variable directa esps. Esto significa que esps explica la mismo en una empresa mediana. Pero la interpretación es distinta con la variables indirectas.

*   En este caso TODAS nuestras variables tienen |t values| mayores que 2. TODAS NUESTRAS VARIABLES SON SIGNIFICATIVAS para explicar el modelo.

Ecuación de regresión por cada tamaño de empresa:
*   Empresa mediana:    

**f1ry = (0.081     + 0.54*epsp -0.16*bmr + 0.04*ato + 0.001*pm)**
*   Empresa pequeña:

f1ry = (0.081-0.059) + (0.54+0.13)*epsp -0.16*bmr + 0.04*ato + 0.001*pm

**f1ry = (0.022     + 0.67*epsp -0.16*bmr + 0.04*ato + 0.001*pm)**
*   Empresa grande:

f1ry = (0.081+0.03)  + (0.54-0.13)4*epsp -0.16*bmr + 0.04*ato + 0.001*pm
     
**f1ry = (0.111     + 0.41*epsp -0.16*bmr + 0.04*ato + 0.001*pm)**



Sobre las variables de interacción:
* Podemos decir que la variable de interacción de una empresa pequeña si es estadisticamente significativa y hace referencia a que, a comparación de la empresa mediana, una empresa pequeña tiene mayor earnings per share deflated by stock price. Dado que epsp es la variable con mayor t value, mayor coeficiente y su intervalo de confianza es positivo podemos afirmar que: **Una empresa pequeña tiene más posibilidad de tener un mayor crecimiento en el retorno trimestral proyectado a un año a comparación de una mediana o una grande de acuerdo a este modelo.**
* Por otra parte la variable de interacción de la empresa grande no es, ni estadisticamente significativa ni indica un claro comportamiento de epsp respecto a la empresa mediana.


#### Versión 4: Regresión con winsorización, trada multicolinealidad y variables de interaccion esps con datos del último año.


In [ ]:
df_winsorized_multicol.columns

In [ ]:
df=df_regression.copy()
#Filtro de años fiscal
df=df[df['qy']=='2022']
df.describe()

In [ ]:
df=df_regression.copy()
#Filtro de años fiscal
df=df[df['qy']=='2022']

#Añadimos variables de interacción
df['epsp*big']=df['epsp']*df['size_big']
df['epsp*small']=df['epsp']*df['size_small']
#Wizorisation
analized=['size_big','size_small','pm','ato','epsp','epsp*big','epsp*small','bmr','f1ry']
df=df[analized]
columns_winzorization=['pm','ato','epsp','bmr']
low_quartile=         [0.02, 0.01,  0.01,  0.01]
high_quartile=        [0.99, 0.99,  0.99,  0.99]
for i in range(len(columns_winzorization)):
  percentile_1 = df[columns_winzorization[i]].quantile(low_quartile[i])
  percentile_99 = df[columns_winzorization[i]].quantile(high_quartile[i])
  df[columns_winzorization[i]]=df[columns_winzorization[i]].clip(lower=percentile_1, upper=percentile_99)
df.describe()
##Guardemos el dataset en df_regression_2022
df_regression_2022=df

In [ ]:
selected=['size_big','size_small','pm','ato','epsp','epsp*big','epsp*small','bmr']
X = df[selected]
# Add a constant term (intercept) to the features
X = sm.add_constant(X)
X.fillna(X.mean(), inplace=True)
# Define your target variable (dependent variable)
y = df["f1ry"]
# Create and fit the linear regression model
model = sm.OLS(y, X, missing="drop").fit()

In [ ]:
# Get the model summary
model_summary = model.summary()
# Print the model summary
print(model_summary)

RESUMEN:
*   Una R^2 de 0.45 ha mejorado por mucho los modelos anteriores.
*   El tmaño de las empresas y el epsp son variables que explican con mayor confianza el modelo.

Ideas:
*   Con argumento estadistico una empresa pequeña gana menos que una empresa mediana y una grande gana más que una mediana en las mismas condiciones. Esto debido a que la constante b es menor, cuando una empresa es pequeña y mayor cuando es grande.
*   Así como en los modelos anteriores a mayor crecimiento de epsp más grandes son los retornos trimestrales en un año futuro.

Hasta el momento este fue el mejor modelo, hicimos limpieza de valores extremos,, agregamos una variable categórica de acuerdo al tamaño,  eliminamos multicolinealidad y añadimos una variable de interacción con nuestra variable con mayor significancia en el modelo.

### 1.2.5 Regression Diagnosis


In [ ]:
influence = model.get_influence()
# Get the summary frame
summary_frame = influence.summary_frame()
# Display the summary frame
print(summary_frame[['cooks_d','standard_resid','hat_diag','student_resid']])

#### Coeficientes con algebra matricial


#### Hat Matrix con algebra matricial

#### Leverage observations


#### Outliers usando studentized residuals

####  outliers usando Cook’s distance


Using matrix algebra calculate the Hat Matrix





In [ ]:

#OLSInfluence(model).summary_frame()